# El seductor canto de las sirenas

In [ ]:
# dependencias
import numpy  as np
import pandas as pd
from   sklearn.model_selection import train_test_split
from   seaborn                 import scatterplot, pairplot
from   sklearn.preprocessing   import StandardScaler, PolynomialFeatures, OneHotEncoder
from   sklearn.model_selection import GridSearchCV
from   sklearn.linear_model    import LogisticRegression
from   sklearn.svm             import LinearSVC
from   sklearn.metrics         import make_scorer, f1_score

In [ ]:
# data paths
path_sirenas_historico    = 'work/datasets/sirenas_endemicas_y_sirenas_migrantes_historico.csv'
path_sirenas_nuevas       = 'work/datasets/sirenas_endemicas_y_sirenas_migrantes.csv'
# output
path_sirenas_predicciones = 'work/datasets/sirenas_endemicas_y_sirenas_migrantes_con_predicciones.csv'

In [ ]:
sirenas_historico = pd.read_csv(path_sirenas_historico)
sirenas_nuevas    = pd.read_csv(path_sirenas_nuevas)

## Exploración inicial

In [ ]:
# primeros y últimos registros
print(sirenas_historico.head())
print(sirenas_historico.tail())

In [ ]:
# estadística descriptiva
sirenas_historico.describe()

In [ ]:
# sirenas por especie
sirenas_historico.groupby(['especie']).agg(['count'])

La cantidad de sirenas esta perfectamente balanceada.

## Separación en train and test
Tenemos 100 observaciones, las clases están balanceadas, ambas tienen `50%` de las observaciones; usaré el `80%` de las observaciones para encontrar el mejor modelo predictivo, y el `20%` restante para medir su desempeño esperado en el mundo real.

In [ ]:
## llevo los datos a un formato para scikit-learn
# definición de las columnas
columnas_predictoras = ['v1', 'v2', 'v3', 'v4']
columna_objetivo     = ['especie']
# separación 
predictores = sirenas_historico[columnas_predictoras]
objetivo    = sirenas_historico[columna_objetivo]
# train_test_split
(x_train, x_test, y_train, y_test) = train_test_split(predictores, objetivo, random_state = 42, test_size = 0.2, shuffle = True, stratify = objetivo)

In [ ]:
# gráficas de dispersión del conjunto de entrenamiento
pairplot(pd.concat([x_train, y_train], axis = 1, ignore_index = True), hue = 4)

##
Las gráficas de dispersión sugieren que las dos clases de sirenas, endemicas y migrantes, son linealmente separables.

## Ingeniería de variables
Estandarizaré las variables, luego añadire las interacciones entre las variables; para la variable objetivo mapearé las sirenas endemicas a `0` y las migrantes a `1`, el formato esperado por scikit-learn (y las mayorias de las librerias).

Probaré dos versiones de los datos preparados, una con solo las variables estandarizadas, y otra con las interacciones añadidas.

In [ ]:
# estandarización: después de esta transformación las variables tendrán promedio 0 y desviación estandar 1
# ajuste
estandarizador = StandardScaler(with_mean = True, with_std = True).fit(x_train)
# transformación
x_train_estandarizados = estandarizador.transform(x_train)

In [ ]:
# añado las interacciones
# ajuste
poly = PolynomialFeatures(interaction_only = True, include_bias = False).fit(x_train_estandarizados)
# transformación
x_train_con_interacciones = poly.transform(x_train_estandarizados)

In [ ]:
# preparación del la variable objetivo, espera 0 o 1, transformaré las endemicas a 0  y las migrantes a 1
# y_train_encoded = encoder.transform(y_train)
codificacion_objetivo = {'especie': {'sirena_endemica': 0, 'sirena_migrante': 1}}
y_train_encoded       = y_train.replace(codificacion_objetivo)

## Modelado
Tenemos un problema de clasificación binaria, con ambas clases perfectamente balanceadas (50% de cada una). Probaré una **regresión logística** clásica, y una **máquina de vectores de soporte (support-vector machine)**; además, usaré validación cruzada y exploraré el espacio de los modelos.  

Dado qué las clases están balanceadas, usaré el **f1 score**, la media armónica entre precisión y exhaustividad, como métrica para determinar el mejor modelo prédictivo (más es mejor); en algunas circunstancias la precisión o la exhaustividad tendrían más sentido, e.g. una especie es más agresiva hacía la otra, pero dado la información disponible el **f1 score** es una buena opción.

In [ ]:
# evaluador para la exploración del espacio de modelos
scorer_f1  = make_scorer(f1_score)

### Regresión logística

In [ ]:
# hiperparametros a explorar
parameters_logit = {'penalty': ['l1', 'l2', 'elasticnet', 'none'], 'C': [0.1, 0.5, 1, 5, 10]}

In [ ]:
# versión sin interacciones
# declaro el modelo
modelo_logit_estandarizado = GridSearchCV(LogisticRegression(solver = 'saga', random_state = 42, n_jobs = 1), param_grid = parameters_logit, n_jobs = 6, cv = 4, scoring = scorer_f1)
# ajusto el modelo
modelo_logit_estandarizado.fit(X = x_train_estandarizados, y = y_train_encoded)
# extraigo el mejor modelo
modelo_logit_estandarizado_mejor = modelo_logit_estandarizado.best_estimator_
# vistazo rápido
print(modelo_logit_estandarizado_mejor)
print(modelo_logit_estandarizado.best_score_)

In [ ]:
# regresión logística con interacciones
# declaro el modelo
modelo_logit_interacciones = GridSearchCV(LogisticRegression(solver = 'saga', random_state = 42, n_jobs = 1), param_grid = parameters_logit, n_jobs = 6, cv = 4, scoring = scorer_f1)
# ajusto el modelo
modelo_logit_interacciones.fit(X = x_train_con_interacciones, y = y_train_encoded)
# extraigo el mejor modelo
modelo_logit_interacciones_mejor = modelo_logit_interacciones.best_estimator_
# vistazo rápido
print(modelo_logit_interacciones_mejor)
print(modelo_logit_interacciones.best_score_)

### Máquina de vectores de soporte (support-vector machine)

In [ ]:
# hiperparametros a probar
parameters_svm = {'penalty': ['l1', 'l2'], 'C': [0.1, .5, 1, 5, 10]}

In [ ]:
# svm sin interacciones
# declaro el modelo
modelo_svm_estandarizado = GridSearchCV(LinearSVC(random_state = 42), param_grid = parameters_svm, n_jobs = 6, cv = 4, scoring = scorer_f1)
# ajusto el modelo
modelo_svm_estandarizado.fit(X = x_train_estandarizados, y = y_train_encoded)
# extraigo el mejor modelo
modelo_svm_estandarizado_mejor = modelo_svm_estandarizado.best_estimator_ 
# vistazo
print(modelo_svm_estandarizado_mejor)
print(modelo_svm_estandarizado.best_score_)

In [ ]:
# svm con interacciones
# declaro el modelo
modelo_svm_interacciones = GridSearchCV(LinearSVC(random_state = 42), param_grid = parameters_svm, n_jobs = 6, cv = 4, scoring = scorer_f1)
# ajusto el modelo
modelo_svm_interacciones.fit(X = x_train_con_interacciones, y = y_train_encoded)
# extraigo el mejor modelo
modelo_svm_interacciones_mejor = modelo_svm_interacciones.best_estimator_ 
# vistazo
print(modelo_svm_interacciones_mejor)
print(modelo_svm_interacciones.best_score_)

## Selección del modelo

De las gráficas de dispersión intuí que las dos clases eran linealmente separables, las 4 combinaciones, 2 modelos y 2 entradas diferentes, que probé lograron un desempeño perfecto, `f1 score = 1`; elegiré la aproximación más sencilla: la regressión logística con la entrada sin interacciones.

## Evaluación del mejor modelo 
Obtendré el error de generalización, usando el mejor modelo predictivo en los datos de test, y comparando esto con los valores reales

In [ ]:
# preparo la entrada para la predicción
# estandarización
x_test_estandarizados = estandarizador.transform(x_test)

In [ ]:
# predicción
prediccion = modelo_logit_estandarizado_mejor.predict(x_test_estandarizados)

In [ ]:
# obtengo el error de generalización
y_test_encoded = y_test.replace(codificacion_objetivo).reset_index(drop = True)
#
error_generalizacion_f1_score = f1_score(y_test_encoded, prediccion)
# print
error_generalizacion_f1_score

Las predicciones son perfectas!!

Hay al menos dos opciones, una es que las dos especies de sirenas son linealmente separanles, o que sobreajustamos los modelos; seguí una metodología sólida, así que confiaré en el modelo predictivo.

## Clasificación de individuos

In [ ]:
# transformo la entrada para la prediccion
sirenas_nuevas_estandarizadas = estandarizador.transform(sirenas_nuevas[columnas_predictoras])

In [ ]:
# uso el mejor modelo para predecir
predicciones_sirenas_nuevas = modelo_logit_estandarizado_mejor.predict(sirenas_nuevas_estandarizadas)
# vistazo
print(predicciones_sirenas_nuevas)

## Salida
Añadiré las predicciones a la tabla

In [ ]:
# copio el dataframe
df_salida = sirenas_nuevas.copy()
# añado las predicciones
df_salida['especie'] = predicciones_sirenas_nuevas
# vistazo
print(df_salida)

In [ ]:
# regreso a las etiquetas originales
codificacion_objetivo_inversa = {'especie': {0: 'sirena_endemica', 1: 'sirena_migrante'}}
# remplazo
df_salida_con_etiquetas = df_salida.replace(codificacion_objetivo_inversa)

In [ ]:
# salvado como csv
df_salida_con_etiquetas.to_csv(path_sirenas_predicciones, index = False)

## Conclusiones